---
layout: post
title: Data storytelling, visualizando datos sobre reservas de hoteles.
date: 2025-05-11 10:49:19
author: Carlos Andrés Moreno
summary: En este artículo vamos a realizar un storytelling de un conjunto de datos de reservas de hoteles
categories: Data
thumbnail: plot
tags:
  - Data
  - Python
  - Plotly
---

# Introducción

Como parte de la Maestría en Cienicas de Datos que estoy estudiando en la [Universidad Oberta de Catalunya](https://uoc.edu), me fue asignada una actividad que consiste en crear un *Data Storytelling* de un conjunto de datos sobre reservaciones de hoteles en Lisboa, Portugal. Creo que es una excelente oportunidad de mostrar un poco lo que he aprendido este semestre en la asignatura de Visualización de Datos, es por eso que he decidido crear el *storytelling* en un nuevo artículo del blog.


# Sobre los datos

El conjunto de datos que vamos a usar a lo largo del *storytelling* ha sido descrito completamente en el artículo titulado [Hotel booking demand datasets ](https://www.sciencedirect.com/science/article/pii/S2352340918315191). Como se ha mencionado anteriormente, los datos tratan sobre reservaciones de hoteles en Lisboa, Portugal hechas desde Julio 1 de 2015 hasta Agosto 31 de 2017. Particularmente dos hoteles, uno ubicado en la zona urbana de la ciudad y otro hotel tipo resort.

Cada fila del conjunto de datos es una reservación hecha por un cliente para alguno de los dos hoteles. Además, tenemos información sobre el número de niños, bebés y adultos dentro de la reserva; més, semana y día en el cual se hizo la reserva; país de residencia de cliente, si la reserva se canceló o no, el tipo de comida que se incluyó para los clientes, entre otra información de interés.

# Herrmientas usadas para la visualización

Todas las visualizaciones que vas a ver han sido creadas usando la librería [Plotly](https://plotly.com/python/). El blog está hosteado en Github pages y utiliza Jekyll para generar todo el sitio (si quieres saber un poco más a cerca de esto, puedes hacer click [acá](https://carmoreno.com.co/tutoriales/2015/08/13/Como-hice-el-blog/)).

Me gustaría tambien mencionar a [Jérome Eertmans](https://github.com/jeertmans), el artículo de su blog titulado ["Embedding Plotly in a Jekyll Post"](https://eertmans.be/posts/plotly-example/) fue de gran ayuda para poder integrar Plotly y Jekyll sin morir en el intento (a veces una buena googleada es mejor que escribirle a Gemini o ChatGPT).

# Ahora sí, manos a la obra.

### ¿Prefieres mucha o poca gente?

Lisboa es uno de los destinos turísticos más visitados todos los años en Europa, tiene un clima excelente, buena comida, hermosos paisajes y playas mediterraneas. En 2021, esta ciudad ocupó [el tercer lugar](https://elordenmundial.com/mapas-y-graficos/ciudades-mas-visitadas-europa/) entre las 15 ciudades más visitadas de Europa, solo por detrás de París y Barcelona. Sin embargo, que sea tan popular no significa que todo sea color de rosa; muchas veces implica sitios de interés repletos de gente, pocos lugares para parquear tu vehículo y pagar precios más altos por temporada. ¿No sería mejor ir a Lisboa en una temporada un poco mas relajada?, podríamos usar los datos de hoteles para saber cuándo hay mayor cantidad de turistas. Entre más reservaciones sin cancelación existan, significa mayor cantidad de personas yendo a estos hoteles. Miremos qué nos dicen los datos.

In [19]:
import polars as pl
import numpy as np
from plotting import express as px
from plotting import graph_objects as go
from plotly.subplots import make_subplots

In [20]:
reservas = pl.read_csv("/Users/carlosm/Documents/MDS/4th Semester/Viusalizacion/PEC3/hotel_bookings.csv", null_values=["NA"])
reservas = reservas.with_columns(
    pl.concat_str(
        pl.col("arrival_date_year"),
        pl.col("arrival_date_month"),
        pl.lit(1),
        separator="-"
    ).str.strptime(pl.Date, "%Y-%B-%d").alias("arrival_date")
)

In [21]:
reservas_por_mes = (
    reservas
        .filter(pl.col("is_canceled") == 0)
        .select(pl.col("arrival_date").dt.month().alias("month"), "hotel", "adr", "arrival_date_month", "country", "adults", "children", "is_canceled")
)
reservas_por_mes

month,hotel,adr,arrival_date_month,country,adults,children,is_canceled
i8,str,f64,str,str,i64,i64,i64
7,"""Resort Hotel""",0.0,"""July""","""PRT""",2,0,0
7,"""Resort Hotel""",0.0,"""July""","""PRT""",2,0,0
7,"""Resort Hotel""",75.0,"""July""","""GBR""",1,0,0
7,"""Resort Hotel""",75.0,"""July""","""GBR""",1,0,0
7,"""Resort Hotel""",98.0,"""July""","""GBR""",2,0,0
…,…,…,…,…,…,…,…
8,"""City Hotel""",96.14,"""August""","""BEL""",2,0,0
8,"""City Hotel""",225.43,"""August""","""FRA""",3,0,0
8,"""City Hotel""",157.71,"""August""","""DEU""",2,0,0


In [22]:
numero_reservas_por_mes = (
    reservas_por_mes
    .group_by("arrival_date_month", "month", "hotel")
        .agg(pl.col("adr").count())
        .rename({"adr": "count"})
        .sort(by="month")
)

numero_reservas_por_mes


arrival_date_month,month,hotel,count
str,i8,str,u32
"""January""",1,"""Resort Hotel""",1868
"""January""",1,"""City Hotel""",2254
"""February""",2,"""Resort Hotel""",2308
"""February""",2,"""City Hotel""",3064
"""March""",3,"""City Hotel""",4072
…,…,…,…
"""October""",10,"""City Hotel""",4337
"""November""",11,"""Resort Hotel""",1976
"""November""",11,"""City Hotel""",2696


In [23]:
fig = px.bar(numero_reservas_por_mes, x="arrival_date_month", y="count", color="hotel", title="Cantidad de reservas sin cancelación por mes (2015 - 2017)", opacity=0.6, labels={"hotel": "Hotel", "arrival_date_month": "Mes de llegada", "count": "Cantidad de reservas"})
fig.show()

Al parecer los meses donde más reservaciones hay son Julio y Agosto, tanto para el Hotel Urbano como para el Hotel Resort. Esto no debería de sorprendernos, puesto que son normalmente en los meses de verano donde más personas salen a vacacionar. Sin embargo, algo que sorprende son los números de reservas completadas para el mes de **Junio**, ya que son pocas en comparación con el increible clima que hace en este mes, solo 6,404.

Fijémonos ahora en el mes de Septiembre, si bien no acostumbra a ser extremadamente soleado de inicio a fin, no esta nada mal para tomarse unas vacaciones, aún así el numero de reservas completadas no son tantas (6,392) como en Julio y Agosto. Por lo general, los primeros cuatro meses del año y los últimos tres no tienen el mejor clima, por eso no es de sorprendernos que la cantidad de reservas completadas no sean tan altas. Pareciera que viajar en Junio y Septiembre nos ofrece una relación `clima/cantidad de turistas` bastante interesante. Pero, ¿de qué nacionalidad serían las personas que nos encontraríamos si viajásemos en alguno de esos dos meses? Exploremos.

### Los más viajeros

Primero miremos de forma general de dónde proceden las personas que han hecho una reserva completada (sin cancelación) en alguno de los dos hoteles.

In [24]:
nacionalidades_reservas = (
    reservas_por_mes.select("country", "hotel", "adr")
        .group_by(["country", "hotel"])
        .agg(pl.col("adr").count())
        .rename({"adr": "count"})
        .sort(by="count", descending=True)
)
nacionalidades_reservas

country,hotel,count
str,str,u32
"""PRT""","""City Hotel""",10879
"""PRT""","""Resort Hotel""",10192
"""FRA""","""City Hotel""",7081
"""GBR""","""Resort Hotel""",5923
"""DEU""","""City Hotel""",5012
…,…,…
"""ZMB""","""Resort Hotel""",1
"""UGA""","""Resort Hotel""",1
"""UZB""","""City Hotel""",1


In [25]:
fig = px.bar(nacionalidades_reservas, x="country", y="count", color="hotel", title="Cantidad de reservaciones no cancelandas por país de origen (2015 - 2017)", opacity=0.6, labels={"hotel": "Hotel", "country": "País", "count": "Cantidad de reservas"})
fig.show()

Vemos que tenemos muchos paises (**178** para ser exactos). Tiene sentido que Portugal sea el país desde donde más se hacen reservas a Lisboa seguido de los paises cercanos como España, Francia, Italia y Reino Unido. Este último al igual que Irlanda parecen preferir el Hotel Resort en lugar del Hotel Urbano.

Por el contrario, los tursitas que vienen de los tres primeros países mencionados anteriormente prefieren visitar el Hotel Urbano, al igual que los turistas que proceden de Alemania. Ahora exploremos de qué paises serían los turistas que nos pudieramos encontrar en Lisboa, en caso de que viajaramos en Junio o Septiembre. Vamos a tomar solo paises que tengan 50 personas o más, con el fin de mostrar datos más significativos. Miremos:

In [26]:
nacionalidad_reservas_mes = (
    reservas_por_mes
    .filter(pl.col("arrival_date_month").is_in(["June", "September"]))
    .group_by("arrival_date_month", "country", "hotel")
    .agg(pl.col("adr").count(), pl.col("adults").sum(), pl.col("children").sum())
    .rename({"adr": "count"})
    .filter(pl.col("count") >= 50)
    .sort(by=["arrival_date_month", "count"], descending=True)
)
nacionalidad_reservas_mes = nacionalidad_reservas_mes.with_columns(
    total_people=(pl.col("adults") + pl.col("children"))
)
nacionalidad_reservas_mes
#reservas_por_mes

arrival_date_month,country,hotel,count,adults,children,total_people
str,str,str,u32,i64,i64,i64
"""September""","""PRT""","""City Hotel""",1196,2005,57,2062
"""September""","""GBR""","""Resort Hotel""",568,1089,26,1115
"""September""","""FRA""","""City Hotel""",519,943,11,954
"""September""","""DEU""","""City Hotel""",447,826,17,843
"""September""","""PRT""","""Resort Hotel""",430,797,40,837
…,…,…,…,…,…,…
"""June""","""SRB""","""City Hotel""",68,74,0,74
"""June""","""SWE""","""City Hotel""",68,128,15,143
"""June""","""DEU""","""Resort Hotel""",56,107,2,109


In [35]:
fig = px.treemap(
    nacionalidad_reservas_mes,
    path=[px.Constant("Months"), 'arrival_date_month', "hotel", 'country'],
    values='total_people',
    color='total_people',
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=np.average(
        (nacionalidad_reservas_mes.select('total_people').get_column("total_people")).to_list()
    ),
    title="Cantidad de personas (adultos y niños) en Junio y Septiembre por país y hotel",
    labels={"total_people": "Total Personas"},
)
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
fig.update_traces(
    marker=dict(cornerradius=5),
    hovertemplate='<b>%{label} </b> <br> Total personas: %{value:,}'
)
fig.show()

El anterior gráfico de árbol nos muestra la cantidad de personas por nacionalidad que podemos encontrarnos, sea que vayamos en Junio o en Septiembre y sea que nos quedemos en el Hotel Resort o en el Hotel Urbano. Puedes pasar el cursor por encima de cada cuadro para obtener el total de personas de cada nacionalidad, también hacer clic por ejemplo en el Hotel Urbano del mes de Junio, para navegar directamente a esta información.

Vemos que si nos hospedamos en el Hotel Resort en cualquiera de los dos meses, tendríamos muchas más chances de encontrarnos Británicos que Portugueses. Por el contrario, en el Hotel Urbano nos encontraríamos en mayor medida con Portugueses, Alemanes y Franceses. En Septiembre podríamos encontrarnos con algún Español en el Hotel Urbano. Parece ser que los Británicos tienden a preferir el Hotel Resort.

Del anterior gráfico también podemos observar que habrá un poco más de turistas en Junio que en Septiembre, sin embargo, será el Hotel Urbano quién tendrá mas cantidad de personas sin importar el mes. Por lo tanto, si tu plan es relajarte en un hotel cercano a la playa y no tanto salir a conocer la ciudad llena de turistas, sería un excelente plan ir al Hotel Resort cerca del mar en Junio ya que habrán menos personas, y si hablas Inglés, podrías aprovechar para practicar, ya que la mayoría de ellos serían Británicos.

### ¡Atentos a las cancelaciones!

Ya vimos que Junio y Septiembre pueden ser los mejores meses para viajar si es que estás buscando un equilibrio entre clima y cantidad de turistas, pero quizá no te importe mucho esto último. Quizá lo más importante para tí sea viajar en los meses de verano sea que haya muchos turístas o no; sin embargo no siempre es posible reservar a tiempo y los mejores meses son los primeros en irse. Que tal si miramos la taza de cancelaciones por mes para mirar si tenemos algún chance de "cazar" alguna reserva cancelada en caso de no poder reservar a tiempo, e irnos de vacaciones a Lisboa.

In [28]:
reservas_canceladas = (
    reservas
        .select(
            "arrival_date_month",
            "hotel",
            "country",
            pl.col("arrival_date").dt.month().alias("month"),
            pl.when(pl.col.is_canceled == 1).then(pl.lit("Si")).otherwise(pl.lit("No")).alias("is_canceled"),
        )
)
reservas_canceladas
#total = reservas_canceladas.select("is_canceled").count()

arrival_date_month,hotel,country,month,is_canceled
str,str,str,i8,str
"""July""","""Resort Hotel""","""PRT""",7,"""No"""
"""July""","""Resort Hotel""","""PRT""",7,"""No"""
"""July""","""Resort Hotel""","""GBR""",7,"""No"""
"""July""","""Resort Hotel""","""GBR""",7,"""No"""
"""July""","""Resort Hotel""","""GBR""",7,"""No"""
…,…,…,…,…
"""August""","""City Hotel""","""BEL""",8,"""No"""
"""August""","""City Hotel""","""FRA""",8,"""No"""
"""August""","""City Hotel""","""DEU""",8,"""No"""


In [29]:
reservas_canceladas_por_mes = (
    reservas_canceladas
        .group_by("arrival_date_month", "month", "hotel", "is_canceled")
        .agg(pl.col("country").count())
        .rename({"country": "count"})
        .sort(by="month")
).with_columns(
    percentage=(pl.col("count") / pl.col("count").sum()) * 100
)
reservas_canceladas_por_mes

arrival_date_month,month,hotel,is_canceled,count,percentage
str,i8,str,str,u32,f64
"""January""",1,"""City Hotel""","""No""",2254,1.88793
"""January""",1,"""City Hotel""","""Si""",1482,1.24131
"""January""",1,"""Resort Hotel""","""No""",1868,1.56462
"""January""",1,"""Resort Hotel""","""Si""",325,0.272217
"""February""",2,"""Resort Hotel""","""Si""",795,0.665885
…,…,…,…,…,…
"""November""",11,"""Resort Hotel""","""No""",1976,1.65508
"""December""",12,"""City Hotel""","""Si""",1740,1.457408
"""December""",12,"""Resort Hotel""","""Si""",631,0.52852


In [30]:
# Realiza un conteo por grupos  para saber el chance de que hayan cancelaciones en un mes y tipo de hotel.
# La idea es que por cada grupo (mes, hotel) obtengamos un porcentaje de reservas canceladas y no canceladas
# Por cada grupo (mes, hotel) el total de este porcentaje sera del 100%.
reservas_canceladas_total_por_grupos = (reservas_canceladas_por_mes
    .group_by("arrival_date_month", "month", "hotel")
    .agg(pl.col("count").sum())
    .rename({"count": "total_by_group"})
    .sort("month"))

reservas_canceladas_total_por_grupos = (
    reservas_canceladas_total_por_grupos
        .join(reservas_canceladas_por_mes, on=["arrival_date_month", "month", "hotel"])
        .with_columns(
            percentage=(pl.col("count") / pl.col("total_by_group")) * 100
        )
        .select("month", "arrival_date_month", "hotel", "is_canceled", "count", "percentage")
        .sort(by=["month", "hotel"])
)
reservas_canceladas_total_por_grupos

month,arrival_date_month,hotel,is_canceled,count,percentage
i8,str,str,str,u32,f64
1,"""January""","""City Hotel""","""No""",2254,60.331906
1,"""January""","""City Hotel""","""Si""",1482,39.668094
1,"""January""","""Resort Hotel""","""No""",1868,85.180119
1,"""January""","""Resort Hotel""","""Si""",325,14.819881
2,"""February""","""City Hotel""","""No""",3064,61.711984
…,…,…,…,…,…
11,"""November""","""Resort Hotel""","""No""",1976,81.083299
12,"""December""","""City Hotel""","""Si""",1740,42.110358
12,"""December""","""City Hotel""","""No""",2392,57.889642


In [31]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "sunburst"}, {"type": "sunburst"}]],
    subplot_titles=("Hotel Resort", "Hotel Urbano")
)
for hotel_type, coord in [("Resort Hotel", {"row": 1, "col": 1}), ("City Hotel", {"row": 1, "col": 2})]:
    subplot = px.sunburst(
        reservas_canceladas_total_por_grupos.filter(pl.col("hotel") == hotel_type).sort(by="month"),
        path=['arrival_date_month', "is_canceled"], values='percentage', color="arrival_date_month",
        color_discrete_sequence=px.colors.qualitative.Light24
    )
    subplot.update_traces(hovertemplate="<b>%{parent}</b><br>Porcentaje: %{value:.2f}% <extra></extra>")
    #print(subplot.data[0]["hovertemplate"])
    fig.add_trace(subplot.data[0], **coord)
fig.update_layout(title_text="Taza de cancelación por mes (2015 - 2017)")
fig.show()

En los anteriores gráficos la etiqueta "Sí" tiene el procentaje de reservas canceladas y "No" el porcentaje de reservas completadas. Para cada mes podemos obtener esta información pasando el mouse por encima de algun mes o simplemente haciendo click sobre el mismo. Vemos que para el total de reservas hechas en el Hotel Reort en el mes de Junio, solamente se cancelaron el **33.07%**, mientras que el **66.93%** restante se completaron. En general, parece que las reservas para este tipo de hotel tienen una taza mas pequeña de cancelaciones. Será mejor planear con antelacion el hospedaje en el Hotel Resort y reservar con antelación.

Por otra parte, en el Hotel Urbano, para el mes de Junio tenemos una taza de cancelaciones más elevadas, de un **44.69%**. Imagina que no alcanzaste a reservar un lugar en Lisboa para tus próximas vacaciones, en este escenario parece más probable que puedas aprovechar alguna cancelación hecha en el Hotel Urbano, pero que sea más dificil "cazar" una para el Hotel Resort.

Otro compartamiento interesante que vemos es que en el mes de Enero, donde el Hotel Resort tiene un porcentaje de cancelación mínimo, de hecho es el más bajo de todos con un **14.82%** (¿vacaciones de trabajo en Enero?). Para finalizar y solo por curiosidad, miremos el porcentaje de cancelaciones de reserva por pais de origen.



### ¿Los más viajeros ahora no lo son tanto?

En el siguiente mapa de calor podemos ver la taza de cancelaciones de los diferentes paises que han hecho al menos diez reservas teniendo en cuenta ambos hoteles. La razon para este filtro es que hay muchos paises que tienen solo una reserva cancelada, esto nos daría una taza del 100% de cancelaciones para ese país, lo cual no sería suficientemente significativo.

Podemos hacer _zoom in_ / _zoom out_ para ir apreciando la taza de cancelaciones por continente. Centrandonos en Europa vemos que la taza mas grande de cancelaciones la tiene Portugal. En un principio pensabamos que eran los que mas viajaban, pero al parecer son tambien los que más cancelan reservas. Por otra parte, en el continente Americano, los paises que menos cancelan reservas de alojamiento a Lisboa son México con el **11.76%** y Perú con el **20.79%**.

In [32]:
reservas_total_pais = (
    reservas_canceladas
    .select("country", "is_canceled", "hotel")
    .group_by(["country"])
    .agg(pl.col("hotel").count().alias("count_bookings"))
)
reservas_canceladas_por_pais = (
    reservas_canceladas
    .select("country", "is_canceled", "hotel")
    .filter(pl.col("is_canceled") == "Si")
    .group_by(["country"])
    .agg(pl.col("hotel").count().alias("count_cancels"))
).join(
    reservas_total_pais, on=["country"]
).with_columns(
    percentage=((pl.col("count_cancels") / pl.col("count_bookings")) * 100).round(2)
).filter(pl.col("count_bookings") > 10)
reservas_canceladas_por_pais

country,count_cancels,count_bookings,percentage
str,u32,u32,f64
"""BEL""",474,2342,20.24
"""ISR""",169,669,25.26
"""POL""",215,919,23.39
"""FRA""",1934,10415,18.57
"""GBR""",2453,12129,20.22
…,…,…,…
"""CZE""",37,171,21.64
"""ESP""",2177,8568,25.41
"""ITA""",1333,3766,35.4


In [33]:
fig = px.choropleth(reservas_canceladas_por_pais, locations="country",
                    color="percentage", # lifeExp is a column of gapminder
                    color_continuous_scale=px.colors.sequential.Oranges,
                    labels={"percentage": "Porcentaje de cancelación"},
                    title="Porcentaje de cancelación de reservas (2015 - 2017)")
fig.update_traces(hovertemplate="<b>%{location}<br>Cancelacion: %{z:.}%</b>")
# print(fig.data[0]["hovertemplate"])
# print()
fig.show()

## Conclusiones

Algunas conclusiones (un poco a la ligera), que podemos sacar luego de este _data storytelling_ son:

* Junio y Septiembre ofrecen una buena relación entre clima y número de turistas.
* Es muy dificil poder reservar alojamiento en el Hotel Resort a última hora, por lo que es mejor hacer este proceso con antelación.
* Portugal es el país de donde vienen los turistas que más viajan a Lisboa, pero también de donde más cancelan reservas (En Europa).
* Es más fácil reservar a última hora en el Hotel Urbano.
* Los Británicos prefieren el Hotel Resort, mientras que el resto de países vecinos prefieren el Hotel Urbano.


Muchas gracías por llegar hasta el final de este ejercicio. El código completo de este artículo lo puedes ver en [este notebook](https://github.com/CarMoreno/carmoreno.github.io/blob/dataviz-test/_notebooks/2025-05-11-Data-storytelling-datos-hotel-ploty-python.ipynb).

{% highlight python %}print("Hasta pronto"){% endhighlight %}